In [1]:
import earthaccess
from dist_s1_enumerator.mgrs_burst_data import get_mgrs_table
import pandas as pd
from tqdm import tqdm
from pathlib import Path

/Users/cmarshak/miniforge3/envs/dist-s1-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_mgrs = get_mgrs_table()

In [3]:
earthaccess.login()

# Testing

In [5]:
START_TIME = '2024-12-15'
STOP_TIME = '2025-01-01'

In [38]:
def get_opera_id(query_item) -> str:
    return dict(query_item.__dict__['render_dict'])['meta']['native-id']

def query_dist_hls_cmr(mgrs_tile_id: str, start_time=START_TIME, stop_time=STOP_TIME):
    collection_short_name = 'OPERA_L3_DIST-ALERT-HLS_V1'
    mgrs_bounds = tuple(df_mgrs[df_mgrs.mgrs_tile_id == mgrs_tile_id].total_bounds)
    mgrs_bounds = tuple(float(x) for x in mgrs_bounds)
    datasets_found = earthaccess.search_data(
        short_name=collection_short_name,
        temporal=(start_time, stop_time),
        cloud_hosted=True,
        bounding_box=mgrs_bounds
    )
    datasets_found = [d for d in datasets_found if mgrs_tile_id in get_opera_id(d)]
    return datasets_found

In [39]:
q = query_dist_hls_cmr('18SUJ')


In [40]:
def get_processing_time(query_item: earthaccess.DataGranule) -> pd.Timestamp:
    data = query_item.__dict__['render_dict']
    ts = pd.Timestamp(data['umm']['TemporalExtent']['RangeDateTime']['BeginningDateTime'])
    return ts

In [41]:
#dict(qs_ordered[0].__dict__['render_dict'])

In [42]:
qs_ordered = sorted(q, key=get_processing_time)

In [43]:
# earthaccess.download(q[0], '.')

# Automate

In [44]:
df

,zip_url,browse_url,product_request_time,processing_duration,high_confidence_alert_threshold,mgrs_tile_id,post_date_buffer_days,stride_for_norm_param_estimation,n_workers_for_norm_param_estimation,delta_lookback_days_mw,...,model_source,memory_strategy,batch_size_for_norm_param_estimation,post_date,track_number,low_confidence_alert_threshold,n_workers_for_despeckling,device,max_pre_imgs_per_burst_mw,model_compilation
0,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,1025.949,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-08-15,32,2.5,4,best,none,False
1,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,1125.674,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-10-02,32,2.5,4,best,none,False
2,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,809.899,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-05-28,105,2.5,4,best,none,False
3,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,627.532,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-10-07,105,2.5,4,best,none,False
4,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-10T16:33:35+00:00,829.556,4.5,50NPL,1,7,4,none,...,transformer_optimized,high,32,2024-11-24,105,2.5,4,best,none,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5298,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-30T18:19:47+00:00,649.291,4.5,52UCF,1,7,4,none,...,transformer_optimized,high,32,2024-04-02,156,2.5,4,best,none,False
5299,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-30T18:19:47+00:00,838.688,4.5,52UCF,1,7,4,none,...,transformer_optimized,high,32,2024-01-09,156,2.5,4,best,none,False
5300,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-30T18:19:47+00:00,599.974,4.5,52UCF,1,7,4,none,...,transformer_optimized,high,32,2024-04-14,156,2.5,4,best,none,False
5301,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-30T18:19:47+00:00,614.636,4.5,52UCF,1,7,4,none,...,transformer_optimized,high,32,2024-05-20,156,2.5,4,best,none,False


In [45]:
x = df.zip_url[0].tolist()

In [46]:
def get_mgrs_tile_id_from_url(url: str):
    zip_path = url.split('/')[-1]
    mgrs_tile_id = zip_path.split('_')[3][1:]
    return mgrs_tile_id

In [47]:
df_0 = pd.read_csv('one_val_product.csv')
df_1 = pd.read_csv('val_products_minus_one.csv')
df = pd.concat([df_0, df_1])
df['mgrs_tile_id'] = df.zip_url.map(get_mgrs_tile_id_from_url)
mgrs_tile_ids = df.mgrs_tile_id.unique().tolist()
mgrs_tile_ids[:3]

['50NPL', '45TUK', '21JTF']

In [51]:
def get_most_recent_date_in_timeframe(mgrs_tile_id: str) -> str:
    datasets = query_dist_hls_cmr(mgrs_tile_id)
    if datasets:
        datasets_ordered = sorted(datasets, key=get_processing_time)
        opera_id = dict(datasets_ordered[-1].__dict__['render_dict'])['meta']['native-id']
        out_dir = Path(f'dist_hls/{mgrs_tile_id}/{opera_id}')
        out_dir.mkdir(exist_ok=True, parents=True)
        r = earthaccess.download(datasets_ordered[-1], out_dir)
        return r
    else:
        return ''

In [52]:
_ = [get_most_recent_date_in_timeframe(m_id) for m_id in tqdm(mgrs_tile_ids)]

  0%|     | 0/85 [00:00<?, ?it/s]
QUEUEING TASKS | : 100%|█| 19/19 

PROCESSING TASKS | :   0%| | 0/19
PROCESSING TASKS | :   5%| | 1/19
PROCESSING TASKS | :  11%| | 2/19
PROCESSING TASKS | :  26%|▎| 5/19
PROCESSING TASKS | :  37%|▎| 7/19
PROCESSING TASKS | :  47%|▍| 9/19
PROCESSING TASKS | :  53%|▌| 10/1
PROCESSING TASKS | :  58%|▌| 11/1
PROCESSING TASKS | :  63%|▋| 12/1
PROCESSING TASKS | :  74%|▋| 14/1
PROCESSING TASKS | :  84%|▊| 16/1
PROCESSING TASKS | : 100%|█| 19/1

COLLECTING RESULTS | : 100%|█| 19
  1%| | 1/85 [00:07<10:22,  7.41s
QUEUEING TASKS | : 100%|█| 19/19 

PROCESSING TASKS | :   0%| | 0/19
PROCESSING TASKS | :   5%| | 1/19
PROCESSING TASKS | :  11%| | 2/19
PROCESSING TASKS | :  21%|▏| 4/19
PROCESSING TASKS | :  32%|▎| 6/19
PROCESSING TASKS | :  37%|▎| 7/19
PROCESSING TASKS | :  42%|▍| 8/19
PROCESSING TASKS | :  47%|▍| 9/19
PROCESSING TASKS | :  58%|▌| 11/1
PROCESSING TASKS | :  68%|▋| 13/1
PROCESSING TASKS | :  74%|▋| 14/1
PROCESSING TASKS | :  84%|▊| 16/1
PROCESSING 